In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import os

In [3]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

0.34.1


In [4]:
Map = geemap.Map(center=[31.87,-80.82], zoom=10)

In [57]:
sc_path = 'F:/US states/SouthCarolina/sc_all.shp'

roi_path = 'F:/DEVELOP/ROI/ROI.shp'

sc_fc = geemap.shp_to_ee(sc_path)

roi_fc = geemap.shp_to_ee(roi_path)
roi = roi_fc.geometry()

Map.addLayer(sc_fc, {}, 'South Carolina')
Map.addLayer(roi, {}, 'ROI - Jasper, Berkeley, Horry Counties')

# Map

In [58]:
def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

In [59]:
#12/08/22 - adding CALIBRATED FLATS
def addFLATSL7(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(RED-SWIR)/(RED+SWIR) - 41.2*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.972*(RED-SWIR)/(RED+SWIR)-0.008) - 41.2*(0.991*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)-0.0014)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))


def addFLATSL8(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.841*(RED-SWIR)/(RED+SWIR) - 0.019) - 41.2*(0.771*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1) + 0.011)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL9(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(1.225*(RED-SWIR)/(RED+SWIR) + 0.096) - 41.2*(1.038* (NIR-RED)/(NIR+6*RED-7.5*BLUE+1) - 0.004)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

##MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.2)
    return image.updateMask(mask1)


##ADDING NDVI (for min/max variables)
def addL5ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'GREEN': image.select('SR_B2')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

def addL8ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'GREEN': image.select('SR_B3')
        })
    
    return image.addBands(ndvi.rename('ndvi'))



In [41]:
#Datasets - NOT MASKING FLATS (try again masking later)
ls5_collect = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterBounds(sc_fc).map(maskL8sr).map(addFLATSL5)
ls7_collect = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterBounds(sc_fc).map(maskL8sr).map(addFLATSL7)
ls8_collect = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(sc_fc).map(maskL8sr).map(addFLATSL8)
ls9_collect = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterBounds(sc_fc).map(maskL8sr).map(addFLATSL8)

#Merge
ultra_col = ls8_collect.merge(ls5_collect).merge(ls7_collect).merge(ls9_collect)

In [42]:
landscan = ee.ImageCollection("projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL").filterBounds(sc_fc)

popcount_intervals = '''
<RasterSymbolizer>
  <ColorMap type="intervals" extended="false">
    <ColorMapEntry color="#CCCCCC" quantity="0" label="No Data"/>
    <ColorMapEntry color="#FFFFBE" quantity="5" label="Population Count (Estimate)"/>
    <ColorMapEntry color="#FEFF73" quantity="25" label="Population Count (Estimate)"/>
    <ColorMapEntry color="#FEFF2C" quantity="50" label="Population Count (Estimate)"/>
    <ColorMapEntry color="#FFAA27" quantity="100" label="Population Count (Estimate)"/>
    <ColorMapEntry color="#FF6625" quantity="500" label="Population Count (Estimate)"/>
    <ColorMapEntry color="#FF0023" quantity="2500" label="Population Count (Estimate)"/>
    <ColorMapEntry color="#CC001A" quantity="5000" label="Population Count (Estimate)"/>
    <ColorMapEntry color="#730009" quantity="185000" label="Population Count (Estimate)"/>
  </ColorMap>
</RasterSymbolizer>
'''

# Define legend colors and labels
legend_dict = {
    'names': [
        '0', '1–5', '6–25', '26–50', '51–100', '101–500', '501–2500',
        '2501–5000', '5001–185000'
    ],
    'colors': [
        '#CCCCCC', '#FFFFBE', '#FEFF73', '#FEFF2C', '#FFAA27',
        '#FF6625', '#FF0023', '#CC001A', '#730009'
    ]
}

# Add legend to the map
Map.add_legend(
    title="Population Count (estimate)",
    labels=legend_dict['names'],
    colors=legend_dict['colors'],
    position='bottomleft'
)


# Add styled layers for 2000 and 2023
pop_2000 = landscan.filter(ee.Filter.calendarRange(2000, 2000, 'year')).first().sldStyle(popcount_intervals).clip(sc_fc)
pop_2023 = landscan.filter(ee.Filter.calendarRange(2023, 2023, 'year')).first().sldStyle(popcount_intervals).clip(sc_fc)

In [43]:
##DAYMET
# daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterBounds(sc_fc).filterDate('2018', '2018')

daymet_og = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterBounds(sc_fc)

#Resampling daymet collection to 30m
def resamp_dm(img):
    resampled_dm = img.reproject(
        crs = 'EPSG:4326',
        scale = 30
    ).resample('bilinear')
    return resampled_dm

resamp_daymet = daymet_og.map(resamp_dm)

daymet = resamp_daymet \
     .filter(ee.Filter.date('2018-09-01', '2018-09-30')) \
     .mean() \
     .clip(sc_fc)

dm_precip = daymet.select('prcp')
dm_maxtemp = daymet.select('tmax')
dm_mintemp = daymet.select('tmin')
dm_daylight = daymet.select('dayl')
dm_srad = daymet.select('srad')
dm_snow = daymet.select('swe')
dm_vapor = daymet.select('vp')

prcpVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

daylVis = {
  'min': 0,
  'max': 50000.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

sradVis = {
  'min': 0,
  'max': 400.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

vpVis = {
  'min': 0,
  'max': 1500,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

In [68]:
dem = ee.Image('USGS/3DEP/10m').clip(roi) ##This is 1/3 arc second, or 10 m.

slope = ee.Terrain.slope(dem)

dem_params = {
    'min': 0,
    'max': 1300,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

In [51]:
l8_param = {'min': 0, 
             'max': 0.2, 
             'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 
             'gamma': 1.5}


# Map.addLayer(ls8_collect.first().select('flats'), {}, 'Landsat 8')
# Map.addLayer(landscan.first().clip(sc_fc), {}, 'Landscan population data')

# Map.addLayer(pop_2000, {}, 'Population Count Estimate 2000')
# Map.addLayer(pop_2023, {}, 'Population Count Estimate 2023')

# Map.addLayer(dm_precip, prcpVis, "Precipitation")

# Map.addLayer(dm_maxtemp, prcpVis, "Maxtemp")
# Map.addLayer(dm_mintemp, prcpVis, "Mintemp")
# Map.addLayer(dm_daylight, daylVis, "Daylight")
# Map.addLayer(dm_srad, sradVis, "SRAD")
# Map.addLayer(dm_snow, prcpVis, "Snowpack water?")
# Map.addLayer(dm_vapor, vpVis, "Vaporpressure")

Map.addLayer(dem, dem_params, 'elevation')

Map

Map(bottom=13348.0, center=[34.379712580462204, -81.11202130498351], controls=(WidgetControl(options=['positio…

No such comm: 2a265326c50b4dd4ac2d823a8e753166
No such comm: 326ee8830b62499b9106552876afe5ea
No such comm: 326ee8830b62499b9106552876afe5ea
No such comm: 555ed812289e4d5bb88ed95d33a59dd1
No such comm: 555ed812289e4d5bb88ed95d33a59dd1


In [66]:
##Exporting GeoTIFFs

# out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
# filename = os.path.join(out_dir, "dem10m_roi.tif")

# geemap.ee_export_image(
#     dem, filename=filename, scale=10, region=roi, file_per_band=True
# )

In [63]:
# Create a grid of tiles (e.g., 10km x 10km)
grid_size = 1000  # Tile size in meters
tiles = geemap.fishnet(roi, rows=10, cols=10, delta = 1)

## Visualize the grid on a map (optional)
# Map = geemap.Map(center=[31.87,-80.82], zoom=10)
# Map.addLayer(roi,{}, 'ROI')
# Map.addLayer(tiles, {}, "Tiles")
# Map

Map(center=[31.87, -80.82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [69]:
# Function to process each tile
def process_tile(tile, tile_id):
    # Clip the image collection to the tile
    slope_clipped = slope.clip(tile)
    
    # Export each tile as GeoTIFF
    filename = os.path.join(out_dir, f'tile_{tile_id}_slope_10m.tif')
    geemap.ee_export_image(
        slope_clipped,
        filename=filename,
        scale=10,
        region=tile,
        file_per_band=True,
    )
    print(f"Exported: {filename}")

# Output directory
out_dir = "F:/DEVELOP/SLOPE/Tiles"
os.makedirs(out_dir, exist_ok=True)

In [71]:
# # Iterate through each tile in the grid
# tile_list = tiles.toList(tiles.size())
# for i in range(tiles.size().getInfo()):
#     tile = ee.Feature(tile_list.get(i)).geometry()
#     process_tile(tile, i)